In [103]:
import geopandas as gpd
import matplotlib
import numpy as np

countries_gdf = gpd.read_file('./Material/ne_10m_admin_0_countries.shp')


# load states from pqrquet
import pandas as pd

df = pd.read_parquet('hdfs://localhost:8020/FinalProject/parquet_archive/date_hour=2019-12-31 23%3A00%3A00/part-00000-fd64f06b-cab9-48f1-981b-06e9e8bfb037.c000.snappy.parquet')
# df = pd.read_json('hdfs://localhost:8020/FinalProject/Archive/opensky_state_vectors_json_2019-12-26_00_04_17.json')

In [89]:
states_gdf = gpd.GeoDataFrame(df[['icao24', 'callsign', 'time', 'on_ground']], geometry = gpd.points_from_xy(df['longitude'], df['latitude']))
states_gdf = states_gdf[states_gdf['geometry'].is_valid]
# c = countries_gdf['NAME'][countries_gdf['geometry'].contains(states_gdf['geometry'][3])]


In [99]:
def get_country(row):
    if not row['geometry'].is_valid:
        print('invalid')
        return ""
    print(type(row['geometry']))
    country = countries_gdf['NAME'][countries_gdf['geometry'].contains(row['geometry'])]
    # Need to handle cases when no country found
    country = country.item() if len(country) > 0 else ""
#     print(f"{row['icao24']} : {country}")
    return country




In [100]:

import numpy as np
from multiprocessing import  Pool

sample = states_gdf[:100]

def parallelize_dataframe(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

def gpd_apply(df):
    df['country'] = df.apply(get_country, axis=1)
    return df



In [101]:
# %%time
res = parallelize_dataframe(sample, gpd_apply)

res.head()

<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.

,icao24,callsign,time,on_ground,geometry,country
0,ab1644,UAL2256,2019-12-31 21:18:40,0,POINT (-100.36680 30.94830),United States of America
1,ab6fdd,AAL233,2019-12-31 21:18:40,0,POINT (-83.09850 28.53740),
2,aaf633,UAL263,2019-12-31 21:18:40,0,POINT (-121.30250 45.07980),United States of America
3,a5e525,N47918,2019-12-31 21:18:40,0,POINT (-86.24820 30.28670),
4,a4b21b,UAL1689,2019-12-31 21:18:40,0,POINT (-118.11360 33.63860),


In [87]:
%%time
sample = states_gdf[:2000]

sample['country'] = sample.apply(get_country, axis=1)

sample.head()

CPU times: user 1min 34s, sys: 492 ms, total: 1min 34s
Wall time: 1min 35s


/home/naya/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [97]:
df.head()

,time,icao24,callsign,last_contact,longitude,latitude,baro_altitude,on_ground,velocity,geo_altitude,squawk,position_source
0,2019-12-31 21:18:40,ab1644,UAL2256,2019-12-31 21:18:40,-100.366798,30.948299,7315.200195,0,210.479996,7505.700195,2524,0
1,2019-12-31 21:18:40,ab6fdd,AAL233,2019-12-31 21:18:40,-83.098503,28.537399,10058.400391,0,265.850006,10515.599609,1353,0
2,2019-12-31 21:18:40,aaf633,UAL263,2019-12-31 21:18:40,-121.302498,45.079800,6789.419922,0,236.509995,6918.959961,6622,0
3,2019-12-31 21:18:40,a5e525,N47918,2019-12-31 21:18:40,-86.248199,30.286699,1341.119995,0,40.389999,1356.359985,2075,0
4,2019-12-31 21:18:40,a4b21b,UAL1689,2019-12-31 21:18:40,-118.113602,33.638599,10660.379883,0,239.720001,10767.059570,3710,0


In [7]:
res.head(150)

,icao24,callsign,time,on_ground,geometry,country
0,ab1644,UAL2256,2019-12-31 21:18:40,0,POINT (-100.36680 30.94830),United States of America
1,ab6fdd,AAL233,2019-12-31 21:18:40,0,POINT (-83.09850 28.53740),
2,aaf633,UAL263,2019-12-31 21:18:40,0,POINT (-121.30250 45.07980),United States of America
3,a5e525,N47918,2019-12-31 21:18:40,0,POINT (-86.24820 30.28670),
4,a4b21b,UAL1689,2019-12-31 21:18:40,0,POINT (-118.11360 33.63860),
5,a3a575,JBU2738,2019-12-31 21:18:40,0,POINT (-68.01480 18.20040),
6,ac4970,RVF960,2019-12-31 21:18:40,0,POINT (-150.73869 60.83490),United States of America
7,a1befc,UAL2025,2019-12-31 21:18:40,0,POINT (-106.33330 39.61890),United States of America
8,aa0967,CFS7676,2019-12-31 21:18:40,0,POINT (-150.04179 61.24890),
9,ad5623,AAL2354,2019-12-31 21:18:40,0,POINT (-105.32410 32.80980),United States of America


In [8]:
!ls ./Material/ne_10m_admin_0_countries.shp

countries.geojson		      ne_10m_admin_0_countries.shx
ne_10m_admin_0_countries.cpg	      ne_10m_admin_0_countries.VERSION.txt
ne_10m_admin_0_countries.dbf	      ne_10m_admin_0_countries.zip
ne_10m_admin_0_countries.prj	      ne_10m_airports.README.html
ne_10m_admin_0_countries.README.html  ne_10m_airports.shp
ne_10m_admin_0_countries.shp	      ne_10m_airports.zip


In [77]:
class A:
    def __init__(self):
        print('class A init')
    
    @staticmethod
    def static_func():
        return(f'A class : static func')
        
    def object_func(self):
        print('A class : not static')

class B(A):
    def object_func(self):
        print
        
    def use_static(self):
        st = super().static_func()
        print(f'B class : using static from A: {st}')

In [78]:
A.static_func()

'A class : static func'

In [79]:
a = A()
b = B()
b.use_static()

class A init
class A init
B class : using static from A: A class : static func


TypeError: getlist() missing 1 required positional argument: 'self'